# Exploring conformational space of selected macrocycles - "M7" 

In this notebook we present and analyze selected structures, technical notes are [here](www.gitlab.com/user/gosia/icho).

In [42]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [43]:
import glob
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Tue Apr  4 10:47:04 2017


In [44]:
# Functions used in this notebook:

def grep_energies_from_sdf_outputs(files):
    energies = {}
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies[os.path.splitext(os.path.basename(inp))[0]] = float(lines[i+1])
    return energies

## Crystal structure of "M7" macrocycle

In [45]:
cm7 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m7/m7_crystal.xyz','r').read()
vcm7 = py3Dmol.view(width=400,height=400)
vcm7.addModel(cm7,'xyz')
vcm7.setStyle({'stick':{}})
vcm7.setBackgroundColor('0xeeeeee')
vcm7.zoomTo()
vcm7.show()

In [46]:
# "core" is a part of a molecule, which we wish to be the "most-aligned" among multiple conformers
m7 = Chem.AddHs(Chem.MolFromSmiles('N1C(=O)c2nc(C(=O)NCCCNC(=O)c3nc(C(=O)NCCC1)ccc3)ccc2'))
core_m7 = m7.GetSubstructMatch(Chem.MolFromSmiles('C(=O)c1nc(C=O)ccc1'))

templ_m7 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m7/m7_crystal.sdf')
m7_crystal = templ_m7[0]

## Conformers generated with the Balloon software:

Conformers were generated in two ways (genetic algorithm):

* Starting with the crystal geometry kept as a template, output: "m7_b_crystal" on the left fig. below

* Starting with the SMILES signature of M7 and allowing to "rebuild the geometry" (option --rebuildGeometry), output: "m7_b_smiles" on the right fig. below

In both cases the Balloon software was asked for 50 conformers (with other parameters set to default values) and it found 10 conformers in the former case and 7 in the latter.

In [47]:
inps_m7_b_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m7/balloon/results_starting_from_crystalsdf/*.sdf')

In [48]:
inps_m7_b_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m7/balloon/results_starting_from_crystalsmiles/*.sdf')

In [49]:
e_m7_b_sdf = grep_energies_from_sdf_outputs(inps_m7_b_sdf)
e_m7_b_smi = grep_energies_from_sdf_outputs(inps_m7_b_smi)

In [50]:
%%html
<table>
  <tr>
    <td id="m7_b_crystal" ></td>
    <td id="m7_b_smiles"  ></td>
  <tr>
    <td> m7_b_crystal </td>
    <td> m7_b_smiles  </td>  
  </tr>
</table>

,
m7_b_crystal,m7_b_smiles


In [51]:
# write conformers to dictionaries
allmol_m7_b_sdf = {}
allmol_m7_b_smi = {}
suppl_m7_b_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m7/balloon/m7_crystal_sdfout.sdf')
suppl_m7_b_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m7/balloon/m7_crystal_smilesout.sdf')

for i, mol in enumerate(suppl_m7_b_sdf):
    name = "m7_b_sdf_" + str(i)
    allmol_m7_b_sdf[name] = mol
for i, mol in enumerate(suppl_m7_b_smi):
    name = "m7_b_smi_" + str(i)
    allmol_m7_b_smi[name] = mol    

In [52]:
# align:
for key, mol in allmol_m7_b_sdf.items():
    AllChem.AlignMolConformers(mol,atomIds=core_m7)
for key, mol in allmol_m7_b_smi.items():
    AllChem.AlignMolConformers(mol,atomIds=core_m7)   

In [53]:
# view:
p7_b_handles=[]

p7_b_sdf = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m7_b_sdf.items():
    mb = Chem.MolToMolBlock(mol)
    p7_b_sdf.addModel(mb,'sdf')
p7_b_sdf.setStyle({'stick':{'radius':'0.15'}})
p7_b_sdf.setBackgroundColor('0xeeeeee')
p7_b_sdf.zoomTo()    
p7_b_handles.append(p7_b_sdf)

p7_b_smi = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m7_b_sdf.items():
    mb = Chem.MolToMolBlock(mol)
    p7_b_smi.addModel(mb,'sdf')
p7_b_smi.setStyle({'stick':{'radius':'0.15'}})
p7_b_smi.setBackgroundColor('0xeeeeee')
p7_b_smi.zoomTo()    
p7_b_handles.append(p7_b_smi)

In [54]:
p7_b_handles[0].insert('m7_b_crystal')

In [55]:
p7_b_handles[1].insert('m7_b_smiles')

### pre-screening

Some of the generated conformers are very much alike. To remove potential duplicates which were not "caught" by the Balloon program, we can compare the energies (preoptimized with MM) and the RMSD calculated against a reference structure (here: the crystal structure of M7).

First let's print the energies and RMS values:

In [56]:
allmol_m7_b = {}
allmol_m7_b.update(allmol_m7_b_sdf)
allmol_m7_b.update(allmol_m7_b_smi)

energy_m7_b = {}
energy_m7_b.update(e_m7_b_sdf)
energy_m7_b.update(e_m7_b_smi)

rms_m7_b = {}
for key, mol in allmol_m7_b.items():
    rms_m7_b[key] = AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(m7_crystal))
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(key, energy_m7_b[key], rms_m7_b[key]))

name = m7_b_smi_3, E = 63.179995, RMS = 1.917011
name = m7_b_sdf_4, E = 55.531843, RMS = 1.877492
name = m7_b_sdf_8, E = 57.683850, RMS = 1.363563
name = m7_b_sdf_9, E = 58.104823, RMS = 1.380051
name = m7_b_smi_6, E = 64.617063, RMS = 2.053687
name = m7_b_smi_1, E = 60.504327, RMS = 1.803571
name = m7_b_sdf_5, E = 55.688970, RMS = 2.016376
name = m7_b_sdf_2, E = 52.100671, RMS = 2.007561
name = m7_b_sdf_0, E = 48.372890, RMS = 0.593787
name = m7_b_smi_0, E = 54.675946, RMS = 0.844525
name = m7_b_smi_4, E = 63.756299, RMS = 1.660194
name = m7_b_sdf_7, E = 57.536989, RMS = 1.921691
name = m7_b_sdf_1, E = 51.783164, RMS = 0.209230
name = m7_b_sdf_3, E = 53.693431, RMS = 1.840951
name = m7_b_smi_2, E = 62.770633, RMS = 1.346777
name = m7_b_sdf_6, E = 56.106820, RMS = 1.799433
name = m7_b_smi_5, E = 64.114107, RMS = 1.414668


Then we can introduce some thresholds, for instance:

* if two conformers differ by less than 0.01 in RMS (measured against the reference structure), then select the one with the lower energy

In [57]:
rms_sorted = sorted(rms_m7_b.items(), key=lambda x: x[1])
rms_thresh = 0.01

print("List sorted by RMS:")
for i, t in enumerate(rms_sorted):
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(rms_sorted[i][0], energy_m7_b[rms_sorted[i][0]], rms_sorted[i][1]))

for i, t in enumerate(rms_sorted):
    if i < len(rms_sorted) and i > 0:
        if (rms_sorted[i][1] - rms_sorted[i-1][1]) < rms_thresh:
            print("similar pairs: ", i-1, ": ", rms_sorted[i-1][0], "  and  ", i, ": ", rms_sorted[i][0])
            if energy_m7_b[rms_sorted[i][0]] < energy_m7_b[rms_sorted[i-1][0]]:
                print("removed conformer: ", rms_sorted[i-1][0])
                del energy_m7_b[rms_sorted[i-1][0]]
                del allmol_m7_b[rms_sorted[i-1][0]]
                del rms_m7_b[rms_sorted[i-1][0]]
            else:
                print("removed conformer: ", rms_sorted[i][0])
                del energy_m7_b[rms_sorted[i][0]]
                del allmol_m7_b[rms_sorted[i][0]]                
                del rms_m7_b[rms_sorted[i][0]]

List sorted by RMS:
name = m7_b_sdf_1, E = 51.783164, RMS = 0.209230
name = m7_b_sdf_0, E = 48.372890, RMS = 0.593787
name = m7_b_smi_0, E = 54.675946, RMS = 0.844525
name = m7_b_smi_2, E = 62.770633, RMS = 1.346777
name = m7_b_sdf_8, E = 57.683850, RMS = 1.363563
name = m7_b_sdf_9, E = 58.104823, RMS = 1.380051
name = m7_b_smi_5, E = 64.114107, RMS = 1.414668
name = m7_b_smi_4, E = 63.756299, RMS = 1.660194
name = m7_b_sdf_6, E = 56.106820, RMS = 1.799433
name = m7_b_smi_1, E = 60.504327, RMS = 1.803571
name = m7_b_sdf_3, E = 53.693431, RMS = 1.840951
name = m7_b_sdf_4, E = 55.531843, RMS = 1.877492
name = m7_b_smi_3, E = 63.179995, RMS = 1.917011
name = m7_b_sdf_7, E = 57.536989, RMS = 1.921691
name = m7_b_sdf_2, E = 52.100671, RMS = 2.007561
name = m7_b_sdf_5, E = 55.688970, RMS = 2.016376
name = m7_b_smi_6, E = 64.617063, RMS = 2.053687
similar pairs:  8 :  m7_b_sdf_6   and   9 :  m7_b_smi_1
removed conformer:  m7_b_smi_1
similar pairs:  12 :  m7_b_smi_3   and   13 :  m7_b_sdf_7
re

All conformers are very much alike (the program finds only *syn-syn* conformers), so we can choose for instance:

* m7_b_sdf_0
* m7_b_sdf_1
* m7_b_sdf_9 (similar E as m1_b_sdf_1, but significantly different RMS)
* m7_b_sdf_7
* m7_b_smi_0
* m7_b_sdf_4 (m7_b_sdf_3, m7_b_sdf_6 rejected; similar E, but lower RMS)
* m7_b_sdf_5 (rejected: m7_b_sdf_2, m7_b_sdf_8, m7_b_smi_1)
* m7_b_smi_6 (rejected: m7_b_smi_5, m7_b_smi_2)
* m7_b_smi_3 (rejected: m7_b_smi_4)

Below we will align the selected conformers:

In [58]:
for key, mol in allmol_m7_b.items():
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles('C(=O)c1nc(C=O)ccc1'))
    AllChem.AlignMol(mol,m7_crystal,atomMap=list(zip(core_mol,core_m7)))
    
p_b = py3Dmol.view(width=400,height=400)
for key, mol in allmol_m7_b.items():
    mb = Chem.MolToMolBlock(mol)
    p_b.addModel(mb,'sdf')
p_b.setStyle({'stick':{'radius':'0.15'}})
p_b.setBackgroundColor('0xeeeeee')
p_b.zoomTo()
p_b.show()

### Conformers generated with the RDKit software

RDKit found 6 conformers of similar energy:

In [ ]:
# create a list of all structures to be aligned
inps_m7_rdkit = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m7/rdkit/results_crystal_from_smiles/*.sdf')

In [ ]:
e_m7_rdkit = grep_energies_from_sdf_outputs(inps_m7_rdkit)

In [ ]:
allmol_m7_rdkit = []
suppl_m7_rdkit = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m7/rdkit/result_smiles.sdf')

for mol in suppl_m7_rdkit:
    allmol_m7_rdkit.append(mol)

This time, instead of visualizing all conformers, we will look directly at their alignment:

In [ ]:
# align:
for mol in allmol_m7_rdkit:
    AllChem.AlignMolConformers(mol,atomIds=core_m7)

In [ ]:
# view:
p = py3Dmol.view(width=400,height=400)
#p.removeAllModels()
for mol in allmol_m7_rdkit:   
    mb = Chem.MolToMolBlock(mol)
    p.addModel(mb,'sdf')    
p.setStyle({'stick':{'radius':'0.15'}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

### Summary

todo: align all conformers